In [1]:
import uproot
import numpy as np
import bitstring
import pandas

In [2]:
class Regions:
    '''A simplistic region maker (not realistic)'''
    def __init__(self, eta_max, n_eta, n_phi):
        self.phi = np.linspace(-np.pi, np.pi, n_phi)
        self.eta = np.linspace(-eta_max, eta_max, n_eta)
        
    def iRegion(self, cand):
        '''Get the (iEta, iPhi) of the candidate'''
        iPhi = np.argwhere((cand.phi >= self.phi) & (cand.phi < np.roll(self.phi, -1)))[0][0]
        iEta = np.argwhere((cand.eta >= self.eta) & (cand.eta < np.roll(self.eta, -1)))[0][0]
        return (iEta, iPhi) 
    
    def regionize(self, event_candidates, sort=True):
        '''Get the particles within the regions'''
        region_candidates = [[[] for i in range(len(self.phi))] for j in range(len(self.eta))]
        for candidate in event_candidates:
            iEta, iPhi = self.iRegion(candidate)
            region_candidates[iEta][iPhi].append(candidate)
        if sort:
            for iEta, regions_eta in enumerate(region_candidates):
                for iPhi, region in enumerate(regions_eta):
                    region_candidates[iEta][iPhi] = sorted(region, key=PuppiCand.pt)
        return region_candidates

class DemoRegions(Regions):
    '''An extension of the Regions class closer to the 6 board correlator demonstrator'''
    def __init__(self):
        super().__init__(3.5, 12, 9)
        
    def pack_event(self, pups, startframe=0, mux=False):
        return self.pack_event_mux(pups, startframe) if mux else \
               self.pack_event_nomux(pups, startframe)
        
    def pack_event_nomux(self, pups, startframe=0):
        '''Return frames of a PatternFile for the unregionized event pups.
        Doesn't treate the regions in a specific order, just groups them in appropriate numbers.'''
        zeropup = PuppiCand(0,0,0).toVHex(True)
        pups = np.array(self.regionize(pups, sort=True)).flatten()
        # 18 links per 6 regions (frames), 16 candidates per region
        # 20 appears because of the link_map SLR handling
        muxed_cands = [[0 for i in range(6 * 16)] for j in range(18)]
        # Loop over 'layer 1 boards' (equivalent)
        for i in range(6):
            # Loop over regions in board
            for j in range(18):
                # Loop over particles in region
                pups_r = pups[18 * i + j]
                for k in range(16):
                    if k < len(pups_r):
                        pup = pups_r[k].toVHex(True)
                    else:
                        pup = zeropup
                    muxed_cands[j][16 * i + k] = pup
        for i, f in enumerate(muxed_cands):
            yield frame(f, startframe+i, 6*16)
        
    def pack_event_mux(self, pups, startframe=0):
        '''Return frames of a PatternFile for the unregionized event pups.
           Doesn't treate the regions in a specific order, just groups them in appropriate numbers.
           Muxes 18 particles per region into 6 links over 3 frames'''
        zeropup = PuppiCand(0,0,0).toVHex(True)
        pups = np.array(self.regionize(pups, sort=True)).flatten()
        # 6 links per 6 regions (frames), 18 candidates per region
        muxed_cands = [[zeropup for i in range(6 * 6)] for j in range(18 * 3)]
        # Loop over 'layer 1 boards' (equivalent)
        for i in range(6):
            # Loop over regions in board
            for j in range(18):
                # Loop over particles in region
                pups_r = pups[18 * i + j]
                for k in range(18):
                    if k < len(pups_r):
                        pup = pups_r[k].toVHex(True)
                    else:
                        pup = zeropup
                    muxed_cands[3 * j + k//6][6 * i + k % 6] = pup
        for i, f in enumerate(muxed_cands):
            yield frame(f, startframe+i, 36)
            
class Particle:
    def __init__(self, pt, eta, phi, hexdata=None):
        if hexdata is not None:
            if isinstance(hexdata, str):
                d = bitstring.pack(hexdata)
            elif isinstance(hexdata, int):
                d = bitstring.pack('int:64', hexdata)
            pt = d[0:15].uint / 4
            eta = d[16:25].uint / 10
            phi = d[26:35].uint / 10
        self.pt = pt
        self.eta = eta
        self.phi = phi
        
    def __str__(self):
        return "Particle({}, {}, {})".format(self.pt, self.eta, self.phi)
    
    def __repr__(self):
        return self.__str__()
    
    def iRegion(self, regions):
        return regions.iRegion(self)
        
    def fromUproot(pt_jagged, eta_jagged, phi_jagged, ptcut=None, pclass=None):
        if pclass is None:
            pclass = Particle
        events = []
        for pti, etai, phii in zip(pt_jagged, eta_jagged, phi_jagged):
            event = []
            for ptij, etaij, phiij in zip(pti, etai, phii):
                particle = pclass(ptij, etaij, phiij)
                if ptcut is None:
                    event.append(particle)
                elif particle.pt > ptcut:
                    event.append(particle)
            events.append(event)
        return events
    
class PuppiCand(Particle):
          
    def __str__(self):
        return super().__str__().replace('Particle', 'PuppiCand')
    
    def fromUproot(pt_jagged, eta_jagged, phi_jagged, ptcut=None):
        return Particle.fromUproot(pt_jagged, eta_jagged, phi_jagged, ptcut, PuppiCand)
    
    def pack_event_hr(pups):
        template = "   cand pt     {} eta {:04.2f} phi {:04.2f}  id 0\n"
        for pup in pups:
            yield template.format(pup.pt, pup.eta, pup.phi)
    
    def pt(self):
        return self.pt
    
    def pack(self):
        eta = int(self.eta * 100)
        phi = int(self.phi * 100)
        pt = int(self.pt * 4)
        v = int(pt > 0)
        bs = bitstring.pack('uint:1,uint:27,int:10,int:10,uint:16',v,0,phi,eta,pt)
        return bs.hex
    
    def toVHex(self, valid):
        return str(int(valid)) + 'v' + self.pack()
    
class Jet(Particle):
            
    def __str__(self):
        return super().__str__().replace('Particle', 'Jet')
    
    def fromUproot(pt_jagged, eta_jagged, phi_jagged, ptcut=None):
        return Particle.fromUproot(pt_jagged, eta_jagged, phi_jagged, ptcut, Jet)
    
    def pack_event_hr(pups):
        template = "   jet pt      {} eta {:04.2f} phi {:04.2f}  constituents 0\n"
        for pup in pups:
            yield template.format(pup.pt, pup.eta, pup.phi)
            
def write_event_hr(f, pups, jets):
    f.write("Event with {} candidates, {} jets in the selected region\n".format(len(pups), len(jets)))
    for pup in PuppiCand.pack_event_hr(pups):
        f.write(pup)
    for jet in Jet.pack_event_hr(jets):
        f.write(jet)
    f.write("\n")

In [3]:
def header(nlinks, board='JETS'):
    txt = 'Board {}\n'.format(board)
    txt += ' Quad/Chan :'
    for i in range(nlinks):
        quadstr = '        q{:02d}c{}      '.format(int(i/4), int(i%4))
        txt += quadstr
    txt += '\n      Link :'
    for i in range(nlinks):
        txt += '         {:03d}       '.format(i)
    txt += '\n'
    return txt

def frame(vhexdata, iframe, nlinks):
    assert(len(vhexdata) == nlinks), "Data length doesn't match expected number of links"
    txt = 'Frame {:04d} :'.format(iframe)
    for d in vhexdata:
        txt += ' ' + d
    txt += '\n'
    return txt

def empty_frames(n, istart, nlinks):
    ''' Make n empty frames for nlinks with starting frame number istart '''
    empty_data = '0v0000000000000000'
    empty_frame = [empty_data] * nlinks
    iframe = istart
    frames = []
    for i in range(n):
        frames.append(frame(empty_frame, iframe, nlinks))
        iframe += 1
    return frames

## Read the ntuple

In [11]:
#d = uproot.open('/home/sioni/Work/jets/debugPF_TTbar_PU200.106X_v0.root')['Events']
d = uproot.open('/home/sioni/Work/corrl2-dev-emp/src/GlobalCorrelator/emp_examples/Layer2/testbench/jet/debugPF.root')['Events']

# Extract Pups
pups = d['l1tPFCandidates_l1pfCandidates_Puppi_RESP.']['l1tPFCandidates_l1pfCandidates_Puppi_RESP.obj']
pt = pups['l1tPFCandidates_l1pfCandidates_Puppi_RESP.obj.m_state.p4Polar_.fCoordinates.fPt'].array()
eta = pups['l1tPFCandidates_l1pfCandidates_Puppi_RESP.obj.m_state.p4Polar_.fCoordinates.fEta'].array()
phi = pups['l1tPFCandidates_l1pfCandidates_Puppi_RESP.obj.m_state.p4Polar_.fCoordinates.fPhi'].array()
pups = PuppiCand.fromUproot(pt, eta, phi, 0)

# Extract Jets
jets = d['l1tPFJets_l1PFSeedConeL1Puppi__RESP.']['l1tPFJets_l1PFSeedConeL1Puppi__RESP.obj']
pt = jets['l1tPFJets_l1PFSeedConeL1Puppi__RESP.obj.m_state.p4Polar_.fCoordinates.fPt'].array()
eta = jets['l1tPFJets_l1PFSeedConeL1Puppi__RESP.obj.m_state.p4Polar_.fCoordinates.fEta'].array()
phi = jets['l1tPFJets_l1PFSeedConeL1Puppi__RESP.obj.m_state.p4Polar_.fCoordinates.fPhi'].array()
jets = Jet.fromUproot(pt, eta, phi, 5)

regions = DemoRegions()

## Write an emp-fwk pattern file

In [13]:
f = open('source.txt', 'w')
f.write(header(6*16))
for d in empty_frames(6, 0, 6 * 16):
    f.write(d)
for d in regions.pack_event(pups[0], startframe=6, mux=False):
    f.write(d)
for d in empty_frames(6, 0+6+18, 6 * 16):
    f.write(d)

## Write an HLS simulation input file

In [12]:
f = open('source_hr.txt', 'w')
write_event_hr(f, pups[0], jets[0])
f.close()

## Debug space

### Check the deregionizer

In [ ]:
regionizer_in = pandas.read_csv('../../../../../../proj/jet_sim/Regionizer-Inputs.txt', skiprows=1, delim_whitespace=True)
pups_in = [PuppiCand(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(regionizer_in.pt, regionizer_in.eta, regionizer_in.phi)]

regionparticles = pandas.read_csv('../../../../../../proj/jet_sim/Regionizer-EventParticles.txt', skiprows=1, delim_whitespace=True)
regionparticles = regionparticles[regionparticles.Clock==56]
pups_merged = [PuppiCand(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(regionparticles.pt, regionparticles.eta, regionparticles.phi)]

### Check the jets

In [14]:
jets_p = pandas.read_csv('../../../../../../proj/jet_sim/Jets.txt', skiprows=1, delim_whitespace=True)

In [15]:
jets_p = jets_p[jets_p.Clock == 236]

In [16]:
jets_p = [Jet(pt/4,eta/100,phi/100) for (pt,eta,phi) in zip(jets_p.pt, jets_p.eta, jets_p.phi)]

In [17]:
jets_p

[Jet(207.5, -0.43, 2.29),
 Jet(187.25, -0.35, -0.3),
 Jet(74.25, 0.55, -1.65),
 Jet(58.25, -0.47, 1.21),
 Jet(41.25, -1.56, 2.47),
 Jet(34.0, -1.63, -1.23),
 Jet(30.75, 0.16, -1.79),
 Jet(5.25, -1.5, 0.29)]

In [18]:
jets[0]

[Jet(207.5, -0.4297322928905487, 2.287867784500122),
 Jet(184.75, -0.3465495705604553, -0.3031064569950104),
 Jet(74.25, 0.5565542578697205, -1.7466663122177124),
 Jet(58.25, -0.47186753153800964, 1.1526070833206177),
 Jet(45.5, -1.6351765394210815, -1.213121771812439),
 Jet(30.75, 0.16418975591659546, -1.8553426265716553),
 Jet(29.75, -1.5189815759658813, 2.34698486328125),
 Jet(5.25, -1.5089000463485718, 0.29120001196861267)]